<a href="https://colab.research.google.com/github/MarquesThiago/analist/blob/main/C%C3%B3pia_de_train_framework_spark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# instalar as dependências
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-2.4.4/spark-2.4.4-bin-hadoop2.7.tgz
!tar xf spark-2.4.4-bin-hadoop2.7.tgz
!pip install -q findspark


In [2]:
!sudo apt-get install openjdk-8-jdk-headless -qq > /dev/null
!sudo add-apt-repository ppa:webupd8team/java
!sudo apt-get update
!sudo apt-get install oracle-java8-installer
!export PYSPARK_SUBMIT_ARGS="--master local[3] pyspark-shell"
!sudo apt-get install openjdk-8-jdk

 The Oracle JDK License has changed for releases starting April 16, 2019.

The new Oracle Technology Network License Agreement for Oracle Java SE is substantially different from prior Oracle JDK licenses. The new license permits certain uses, such as personal use and development use, at no cost -- but other uses authorized under prior Oracle JDK licenses may no longer be available. Please review the terms carefully before downloading and using this product. An FAQ is available here: https://www.oracle.com/technetwork/java/javase/overview/oracle-jdk-faqs.html

Oracle Java downloads now require logging in to an Oracle account to download Java updates, like the latest Oracle Java 8u211 / Java SE 8u212. Because of this I cannot update the PPA with the latest Java (and the old links were broken by Oracle).

For this reason, THIS PPA IS DISCONTINUED.

UPDATE:

For Oracle Java 17, see a different PPA -> https://www.linuxuprising.com/2021/09/how-to-install-oracle-java-17-lts-on.html

Old descr

In [3]:
# configurar as variáveis de ambiente
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.4-bin-hadoop2.7"

!pip install -q pyspark
!pip install beautifulsoup4
# tornar o pyspark "importável"
import findspark
findspark.init('spark-2.4.4-bin-hadoop2.7')

     |████████████████████████████████| 281.3 MB 26 kB/s 
     |████████████████████████████████| 199 kB 38.6 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
from pyspark.sql import SparkSession

In [21]:
from pyspark.sql.functions import max, count, col, desc, asc, lit, sum as sumerize, length
import pyspark.sql.functions as F
from pyspark import sql

In [6]:
from pyspark.sql import DataFrame

In [7]:
from pyspark.sql.functions import when, expr

In [8]:

import pandas as pd
import requests as res
from bs4 import BeautifulSoup as bs
from io import StringIO

url = "https://github.com/MarquesThiago/linear_regresion/blob/master/Housing/USA_Housing.csv"
get = res.get(url=url)
soup = bs(get.content, "html.parser")
table = soup.select("table[data-tagsearch-path]")
list_texts = table[0].select(".blob-code")
texts = [text.get_text() for text in list_texts]
file_csv = "\n".join(texts) 
csv = StringIO(file_csv)

test = pd.read_csv(csv, error_bad_lines=False, sep = ",")
test.head()


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)


,Avg. Area Income,Avg. Area House Age,Avg. Area Number of Rooms,Avg. Area Number of Bedrooms,Area Population,Price,Address
0,79545.458574,5.682861,7.009188,4.09,23086.800503,1.059034e+06,"208 Michael Ferry Apt. 674\nLaurabury, NE 3701..."
1,79248.642455,6.002900,6.730821,3.09,40173.072174,1.505891e+06,"188 Johnson Views Suite 079\nLake Kathleen, CA..."
2,61287.067179,5.865890,8.512727,5.13,36882.159400,1.058988e+06,"9127 Elizabeth Stravenue\nDanieltown, WI 06482..."
3,63345.240046,7.188236,5.586729,3.26,34310.242831,1.260617e+06,USS Barnett\nFPO AP 44820
4,59982.197226,5.040555,7.839388,4.23,26354.109472,6.309435e+05,USNS Raymond\nFPO AE 09386


In [9]:
sc = SparkSession.builder.master("local[*]").getOrCreate()


In [10]:

# carregar dados do Airbnb
df_spark = sc.read.csv("/content/sample_data/california_housing_train.csv", inferSchema=True, header=True)
sparkDF=sc.createDataFrame(test) 

# ver algumas informações sobre os tipos de dados de cada coluna
print("primeiro dataframe")
df_spark.printSchema()
print("segundo dataframe")
sparkDF.printSchema()



primeiro dataframe
root
 |-- longitude: double (nullable = true)
 |-- latitude: double (nullable = true)
 |-- housing_median_age: double (nullable = true)
 |-- total_rooms: double (nullable = true)
 |-- total_bedrooms: double (nullable = true)
 |-- population: double (nullable = true)
 |-- households: double (nullable = true)
 |-- median_income: double (nullable = true)
 |-- median_house_value: double (nullable = true)

segundo dataframe
root
 |-- Avg. Area Income: double (nullable = true)
 |-- Avg. Area House Age: double (nullable = true)
 |-- Avg. Area Number of Rooms: double (nullable = true)
 |-- Avg. Area Number of Bedrooms: double (nullable = true)
 |-- Area Population: double (nullable = true)
 |-- Price: double (nullable = true)
 |-- Address: string (nullable = true)



In [11]:
# realizando casting da idede media da casa de ponto flutauante para inteiro
treat_df = df_spark.withColumn('house_age', df_spark.housing_median_age.cast('int')).drop('housing_median_age')
treat_df.show()

+---------+--------+-----------+--------------+----------+----------+-------------+------------------+---------+
|longitude|latitude|total_rooms|total_bedrooms|population|households|median_income|median_house_value|house_age|
+---------+--------+-----------+--------------+----------+----------+-------------+------------------+---------+
|  -114.31|   34.19|     5612.0|        1283.0|    1015.0|     472.0|       1.4936|           66900.0|       15|
|  -114.47|    34.4|     7650.0|        1901.0|    1129.0|     463.0|         1.82|           80100.0|       19|
|  -114.56|   33.69|      720.0|         174.0|     333.0|     117.0|       1.6509|           85700.0|       17|
|  -114.57|   33.64|     1501.0|         337.0|     515.0|     226.0|       3.1917|           73400.0|       14|
|  -114.57|   33.57|     1454.0|         326.0|     624.0|     262.0|        1.925|           65500.0|       20|
|  -114.58|   33.63|     1387.0|         236.0|     671.0|     239.0|       3.3438|           74

In [12]:
# retornado os 10 maiores valores de case 
treat_df.sort(col('median_house_value').desc()).show(10)

+---------+--------+-----------+--------------+----------+----------+-------------+------------------+---------+
|longitude|latitude|total_rooms|total_bedrooms|population|households|median_income|median_house_value|house_age|
+---------+--------+-----------+--------------+----------+----------+-------------+------------------+---------+
|  -117.23|   32.99|     2718.0|         326.0|    1011.0|     319.0|      15.0001|          500001.0|       17|
|  -117.23|   33.01|     3961.0|         511.0|    1541.0|     470.0|      11.1118|          500001.0|       18|
|  -117.19|   32.76|     1294.0|         175.0|     434.0|     180.0|       5.7914|          500001.0|       52|
|  -116.43|   33.75|     2596.0|         438.0|     473.0|     237.0|       3.7727|          500001.0|       24|
|  -117.19|   32.75|     1495.0|         230.0|     459.0|     190.0|       8.1548|          500001.0|       52|
|  -117.08|   34.08|       45.0|          11.0|      39.0|      14.0|       3.0625|          500

In [13]:
treat_df.createOrReplaceTempView("house")

sc.sql("SELECT * FROM house ORDER BY median_income desc limit 10 ").show()

sc.catalog.dropTempView("house")

+---------+--------+-----------+--------------+----------+----------+-------------+------------------+---------+
|longitude|latitude|total_rooms|total_bedrooms|population|households|median_income|median_house_value|house_age|
+---------+--------+-----------+--------------+----------+----------+-------------+------------------+---------+
|  -118.04|   34.13|      249.0|          31.0|     268.0|      29.0|      15.0001|          500001.0|       35|
|  -118.34|   34.08|     1721.0|         195.0|     688.0|     196.0|      15.0001|          500001.0|       52|
|  -118.06|   33.72|     2665.0|         331.0|     964.0|     319.0|      15.0001|          500001.0|       14|
|  -118.19|   34.19|     2061.0|         260.0|     825.0|     254.0|      15.0001|          500001.0|       34|
|  -118.33|   34.06|     1841.0|         240.0|     693.0|     218.0|      15.0001|          500001.0|       52|
|  -118.32|   34.06|      955.0|         100.0|     457.0|     120.0|      15.0001|          500

In [14]:
def faixa_idade(x): 
  if x < 1:
    return "new"
  elif x >= 1 and x < 5:
    return "atual house"
  elif x >= 5 and x < 20:
    return "modern house"
  elif x >= 20 and x < 50:
    return "contemporary house"
  else:
    return "old house"   

In [15]:

df3 = treat_df.select(
    col("*"),
    when(col('house_Age') < 1, 'new')
    .when(
        (col('house_Age') >= 1) & (col('house_Age') < 5), "atual house")
    .when(
        (col('house_Age') >= 5) & (col('house_Age') < 20), "modern house")
    .when(
        (col('house_Age') >= 20) & (col('house_Age') < 50), "contemporary house")
    .otherwise("old house").alias("range_age_house")

)

In [16]:
df3.show(10)

+---------+--------+-----------+--------------+----------+----------+-------------+------------------+---------+------------------+
|longitude|latitude|total_rooms|total_bedrooms|population|households|median_income|median_house_value|house_age|   range_age_house|
+---------+--------+-----------+--------------+----------+----------+-------------+------------------+---------+------------------+
|  -114.31|   34.19|     5612.0|        1283.0|    1015.0|     472.0|       1.4936|           66900.0|       15|      modern house|
|  -114.47|    34.4|     7650.0|        1901.0|    1129.0|     463.0|         1.82|           80100.0|       19|      modern house|
|  -114.56|   33.69|      720.0|         174.0|     333.0|     117.0|       1.6509|           85700.0|       17|      modern house|
|  -114.57|   33.64|     1501.0|         337.0|     515.0|     226.0|       3.1917|           73400.0|       14|      modern house|
|  -114.57|   33.57|     1454.0|         326.0|     624.0|     262.0|       

In [17]:
treat_df.withColumn('range_age_house', expr(
    "CASE "
    +"WHEN house_age = 1 THEN 'new' "
    +"WHEN house_age >= 1 AND house_age < 5 THEN 'atual_house' "
    +"WHEN house_age >= 5 AND house_age < 20 THEN 'modern house' "
    +"WHEN house_age >= 20 AND house_age < 50 THEN 'contemporary house' "
    +"ELSE 'old house' "
    +"END")
).show(10)

+---------+--------+-----------+--------------+----------+----------+-------------+------------------+---------+------------------+
|longitude|latitude|total_rooms|total_bedrooms|population|households|median_income|median_house_value|house_age|   range_age_house|
+---------+--------+-----------+--------------+----------+----------+-------------+------------------+---------+------------------+
|  -114.31|   34.19|     5612.0|        1283.0|    1015.0|     472.0|       1.4936|           66900.0|       15|      modern house|
|  -114.47|    34.4|     7650.0|        1901.0|    1129.0|     463.0|         1.82|           80100.0|       19|      modern house|
|  -114.56|   33.69|      720.0|         174.0|     333.0|     117.0|       1.6509|           85700.0|       17|      modern house|
|  -114.57|   33.64|     1501.0|         337.0|     515.0|     226.0|       3.1917|           73400.0|       14|      modern house|
|  -114.57|   33.57|     1454.0|         326.0|     624.0|     262.0|       

### **FILTER in PYSpark**

In [18]:
df3.filter("range_age_house == 'new'").show()

+---------+--------+-----------+--------------+----------+----------+-------------+------------------+---------+---------------+
|longitude|latitude|total_rooms|total_bedrooms|population|households|median_income|median_house_value|house_age|range_age_house|
+---------+--------+-----------+--------------+----------+----------+-------------+------------------+---------+---------------+
+---------+--------+-----------+--------------+----------+----------+-------------+------------------+---------+---------------+



In [19]:
df3.filter(col("range_age_house") == 'old house').show(10, truncate = False)

+---------+--------+-----------+--------------+----------+----------+-------------+------------------+---------+---------------+
|longitude|latitude|total_rooms|total_bedrooms|population|households|median_income|median_house_value|house_age|range_age_house|
+---------+--------+-----------+--------------+----------+----------+-------------+------------------+---------+---------------+
|-116.88  |34.08   |3419.0     |777.0         |710.0     |265.0     |3.9028       |128600.0          |52       |old house      |
|-117.1   |32.76   |2606.0     |426.0         |883.0     |380.0     |4.2813       |270800.0          |52       |old house      |
|-117.11  |32.77   |1506.0     |233.0         |478.0     |240.0     |4.3875       |300000.0          |52       |old house      |
|-117.11  |32.77   |1484.0     |224.0         |498.0     |223.0     |6.6053       |331400.0          |52       |old house      |
|-117.11  |32.77   |1729.0     |355.0         |617.0     |337.0     |3.6705       |167000.0      

In [22]:
df3.groupBy('range_age_house').agg(
    count("house_age").alias("counter"), 
    sumerize("median_house_value").alias("sum_value_range"),
    sumerize('median_income').alias("sum_income_range")
    ).show()

+------------------+-------+---------------+------------------+
|   range_age_house|counter|sum_value_range|  sum_income_range|
+------------------+-------+---------------+------------------+
|      modern house|   4568|   8.75698001E8|18844.772299999957|
|       atual house|    258|    5.9676416E7|1351.0287000000008|
|         old house|   1196|   3.24548061E8| 4654.436500000001|
|contemporary house|  10978|  2.264193032E9| 41170.59019999987|
+------------------+-------+---------------+------------------+



In [23]:
schema = df_spark.printSchema()

root
 |-- longitude: double (nullable = true)
 |-- latitude: double (nullable = true)
 |-- housing_median_age: double (nullable = true)
 |-- total_rooms: double (nullable = true)
 |-- total_bedrooms: double (nullable = true)
 |-- population: double (nullable = true)
 |-- households: double (nullable = true)
 |-- median_income: double (nullable = true)
 |-- median_house_value: double (nullable = true)



In [24]:
df_spark.columns

['longitude',
 'latitude',
 'housing_median_age',
 'total_rooms',
 'total_bedrooms',
 'population',
 'households',
 'median_income',
 'median_house_value']

SELECT

In [25]:
df_spark.select(col("longitude")).show(5)

+---------+
|longitude|
+---------+
|  -114.31|
|  -114.47|
|  -114.56|
|  -114.57|
|  -114.57|
+---------+
only showing top 5 rows



In [26]:
df_spark.select(df_spark.longitude, df_spark.latitude).show(5)

+---------+--------+
|longitude|latitude|
+---------+--------+
|  -114.31|   34.19|
|  -114.47|    34.4|
|  -114.56|   33.69|
|  -114.57|   33.64|
|  -114.57|   33.57|
+---------+--------+
only showing top 5 rows



In [27]:
df_spark.select(['longitude',
 'latitude',
 'housing_median_age',
 'total_rooms',
 'total_bedrooms',
 'population']).show()

+---------+--------+------------------+-----------+--------------+----------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|
+---------+--------+------------------+-----------+--------------+----------+
|  -114.31|   34.19|              15.0|     5612.0|        1283.0|    1015.0|
|  -114.47|    34.4|              19.0|     7650.0|        1901.0|    1129.0|
|  -114.56|   33.69|              17.0|      720.0|         174.0|     333.0|
|  -114.57|   33.64|              14.0|     1501.0|         337.0|     515.0|
|  -114.57|   33.57|              20.0|     1454.0|         326.0|     624.0|
|  -114.58|   33.63|              29.0|     1387.0|         236.0|     671.0|
|  -114.58|   33.61|              25.0|     2907.0|         680.0|    1841.0|
|  -114.59|   34.83|              41.0|      812.0|         168.0|     375.0|
|  -114.59|   33.61|              34.0|     4789.0|        1175.0|    3134.0|
|   -114.6|   34.83|              46.0|     1497.0|         309.

Lenght

In [32]:
# utlizando a função lenght para medir o tamanho de uma string
te = sparkDF.withColumn("len", length("Address"))

In [56]:
te.select(col("len"),

[Column<b'len'>]


In [ ]:

# carregar dados do Airbnb
df_spark = sc.read.csv("/content/sample_data/california_housing_train.csv", inferSchema=True, header=True)
sparkDF=sc.createDataFrame(test) 

# ver algumas informações sobre os tipos de dados de cada coluna
print("primeiro dataframe")
df_spark.printSchema()
print("segundo dataframe")
sparkDF.printSchema()

 

## MERGE

In [ ]:
df_spark.join(sparkDF, df_spark.median_house_value == sparkDF.Price, "leftouter").show()

In [ ]:
df_spark.join(sparkDF, df_spark.median_house_value == sparkDF.Price, "right").show()